# Vending Machine 
## An in depth explanation of the source code structure
The source code in question is under the name `Mini_Project_Kel_3.py` in this repository. This notebook will explain the structure of the source code in depth, and the reasoning behind the choices made in the implementation of the code.

---

## Defining the specifications of the vending machine program
The vending machine in this project is designed to be a simple simulation of a real-world vending machine with the following characteristics:
1. The vending machine requires a system for managing its inventory of products, including the ability to add, remove, and update products.
> Our intuition to implement this is through the use of python list or dictionaries to enable modular and flexible management of product data, but this project is under the restriction of using **only the material already taught in our lectures** which does not include neither lists or dictionaries. Therefore in order to implement this feature, we will leverage the use of `string` in python to store the product data in a structured format, and use string manipulation methods to read/write the data of the products in the inventory.
2. The vending machine must be able to accept payments in multiple currencies, including IDR and SGD.
> This program should handle transactions in both IDR and SGD, and convert between the two currencies as needed. Although real-time conversion between transactions are not supported due to exchange rates that may cause losses everytime a conversion is made - which realistically is not desired in a realistic business sense - **the conversion rate can be set in the source code** to simulate the ever changing exchange rate of currencies in the real world.
3. The vending machine must be able to dispense change to customers in the same currency as the payment.
> This program should be able to reliably calculate the correct amount of change to dispense to the customer, taking into account the currency used for the transaction. The change calculation should be done using a **greedy algorithm** to minimize the number of banknotes returned to the customer. With the added restriction of a **cash dispense limit of 100 unit for each banknote**, the program should also be able to handle situations where the machine does not have enough banknotes to give the exact change requested by the customer.
4. The source code must be well-structured and easy to read, with clear comments and documentation.
> Despite the lack of formal programming experience among the team members, we will do our best to structure the code in a logical and easy-to-follow manner, with clear comments and documentation to explain the processes behind the implementation of the code.


# STARTING WITH THE FOUNDATION OF THE PRODUCT INVENTORY SYSTEM


In [ ]:
PRODUCTS = '''
-Chatito.10500|0:10******
-El Minerale.3800|0:10***
-ReeCheez.11300|0:10*****
-SUPER FOOD.24700|0:10***
-Cola Coca.5000|0:10*****
-Deez Peas.12300|0:10****
'''
DEFAULT_PRODUCT_STOCK = 5 # THIS IS TO SET THE DEFAULT STOCK OF EVERY PRODUCT ITEM # TWEAK HERE !
DEFAULT_MONEY_STOCK = 100 # THIS IS TO SET THE DEFAULT STOCK OF THE PHYSICAL MONEY # TWEAK HERE !
# say $1 SGD dollar is Rp.12,800 IDR would be :
SGD_to_IDR_ratio = 128.0 # 1 SGD cent is 128 IDR # CHANGE THE RATIO HERE !!!!!!!!!!!!!!!!!!!!!!!!

### At the top of the source code, these global variables can be set by `owners` of the vending machine to customize the product inventory and currency conversion rate of the vending machine.

Instead of using multiple variables to store each data of the products, we implemented a system we call `big_strings`. Such as `PRODUCTS` is a string variable that is designed to behave as a "list" of products. So called `owners` of the vending machine can **easily add or remove products in this big string** with these data by following a strict format where each `line` are products where its data is separated by dashes `-`, dots `.`, vertical bars `|`, colons `:`, and asterisks `*` to separate different data fields. The format is as follows:
```python
PRODUCTS = '''
-Name.IDR|SGD:stock******
'''
```
Where:
- `Name` is the name of the product, which can be any string without the special characters used as separators. Preceded by a dash `-` and followed by a dot `.`
- `IDR` is the price of the product in Indonesian Rupiah, which is an integer without any special characters. Followed by a vertical bar `|`
- `SGD` is the price of the product in Singapore Dollars but **as their cents** to avoid decimals. Followed by a colon `:`
- `stock` is the stock of the product, which is an integer without any special characters. Followed by asterisks `*` as fillers to make the whole line **exactly 25 characters long.**
- At the end of each 25 characters-long line, there is a newline character `\n` to separate each product line. Note that this character is **not displayed** by IDEs or text editors, but it is there.

# Why 25 characters long?
In python, `strings` can be imagined as **chains of individual characters** that are **indexed starting from 0**. So although `PRODUCTS` have lines separated by `\n` newline characters, it is actually a **one dimensional string** of characters. In order to traverse the one dimensional string, **loops would need to iterate through each character one by one** which would cost performance because this would be slow and inefficient for the designed lengths of these big strings. 

Thus, to maintain efficiency, requires a system to **'jump' to the start of each line directly** without having to loop through each character one by one. We call this `line jumping` and this is achieved by making each line **exactly 25 characters long**, so that the starting index of each line can be precisely calculated and thus creating **a reliable line jumping method.**

These big string structures start with a newline character `\n` at index 0, followed by the first line of product data `-` starting at index 1, and 25 characters later `*` ends at index 25, with a newline character `\n` at index 26. The second line of product data `-` starts at index 27, and so on. So with newline characters being represented with these two symbols `\n` but not displayed by IDEs and text editors, you can imagine the `PRODUCTS` big string as follows:
```python
'''\n
-Name.IDR|SGD:stock******\n
-Name.IDR|SGD:stock******\n
'''
```
### The 25 characters length, the separators of each data field, and the newline characters are all **crucial** for the program to function correctly. If any of these are not followed, the program may not be able to read the product data correctly, leading to errors or unexpected behavior.
#### At least one product must be present in the `PRODUCTS` big string for the program to run correctly, with each data field filled with valid data that can just be **placeholders** and the correct order of separators maintained.

# count_lines() function
To facilitate the management of these big strings structures, we implemented a function called `count_lines()` that simply counts each `\n` newline character.

In [ ]:
def count_lines(big_string): # THIS IS TO GIVE COUNT OF HOW MANY ITEMS IN THESE BIG STRINGS
    j = 0
    for i in big_string:
        if i == '\n':
            j += 1
    return j
# product_count stores how many PRODUCTS we have
product_count = count_lines(PRODUCTS)
# product_count will be +1 extra but this can be taken advantage since range(end) is exclusive!
# next this checks if the product big_string is valid or not... this is just in case we made an error
if (len(PRODUCTS) - product_count) % 25 != 0 :
    float("CAUTION !!! THE PRODUCT big_string IS INVALID !!!")

This may be seen as redundant since `product_count` will then have one extra count due to the newline character at the start of the big string, but this is intentional as further down the line accessing each line of the big string will be done by for loops with the `range(start, end)` function, where the `start` is inclusive and the `end` is exclusive. `product_count` will be used as the `end` parameter in the `range()` function, so that the loop will iterate through each line of the big string correctly.   

Next lies a simple check to ensure the big string has the right 25 charaacter-long format, it is not a complete validation process but it is enough to catch most common errors such as missing newline characters or lines that are not 25 characters long. **The responsibility of ensuring the big string is in the correct format lies with the `owners` of the vending machine, as they are the ones who will be editing the big string to add or remove products.**

# The 'Bank' currency system
The vending machine is designed to accept payments in both IDR and SGD, and to dispense change in the same currency as the payment. To facilitate this, we implemented `BANK_IDR` and `BANK_SGD` to represent the banknotes available in the vending machine for each currency. This system is also built upon the same `big string` structure as `PRODUCTS`, with each line representing a banknote denomination and its stock.

In [ ]:
# ------------------------------------------------ THIS IS THE BANK TO STORE THE MACHINE"S CASH
BANK_IDR = '''
-Rp.100_000:000**********
-Rp.50_000:000***********
-Rp.20_000:000***********
-Rp.10_000:000***********
-Rp.5_000:000************
-Rp.2_000:000************
-Rp.1_000:100************
-Rp.500:100**************
-Rp.200:100**************
-Rp.100:100**************
'''
# Notice SGD is stored in cents to avoid decimals because the machine's processes work best with integers
BANK_SGD = '''
-$.100_00:000************
-$.50_00:000*************
-$.10_00:000*************
-$.5_00:000**************
-$.2_00:000**************
-$.1_00:000**************
-$.50:000****************
-$.20:100****************
-$.10:100****************
-$.5:100*****************
-$.1:100*****************
'''
idr_count = count_lines(BANK_IDR) # stores how many IDR banknotes we gonna use
sgd_count = count_lines(BANK_SGD) # stores how many SGD banknotes we gonna use

Here the 25 characters-long format is maintained as well but it does seem to waste a lot of space for just two data fields, but **this is done to maintain consistency across the big string structures in the program**, avoiding the need for special cases in the code to handle different formats for different big strings. 
1. Here the data field stored between `-` and `.` may act as just placeholders that dont actually get processed at all but may serve as identifiers for the banknotes for the `owners` of the vending machine to easily identify each banknote denomination.
2. The data field stored between `.` and `:` are the **banknote denomination**, which is an integer without any special characters. IDR banknotes are in full units of Rupiah, while SGD banknotes are in **cents to avoid decimals.**
3. The data field stored between `:` and `*` is the **stock** of the banknote, which is an integer without any special characters. Followed by asterisks `*` as fillers to make the whole line **exactly 25 characters long.**

# The decision to avoid decimals
Down the line, this program's implementation of the *greedy cashier algortihm* will involve the effective use of modulo `%` operators and floor integer division `//` operators to calculate the number of banknotes needed for a given amount of change. In python, **these operators behave differently and rather unpredictable when applied to floating point numbers (decimals) compared to integers**. Therefore, to avoid potential pitfalls and ensure the reliability of the change calculation process, we decided to represent SGD banknotes in cents (integers) instead of dollars (decimals). This decision simplifies the implementation of the greedy algorithm and minimizes the risk of errors in change calculation due to floating point arithmetic issues.

# FOUNDATIONAL FUNCTIONS THAT MANAGES THIS BIG STRING STRUCTURE

## get_line(line, big_string)
where `line` is an integer that specifies which line in the big string to access, that starts from line 1. And `big_string` is the big string variable to access, such as `PRODUCTS`, `BANK_IDR`, or `BANK_SGD`.
> This function returns the specified `line` from the `big_string`, which is the line of the product's data fields with its separators up until it reaches the first asterisk `*`

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> GET ANY LINE FROM A BIG STRING
def get_line(line, big_string):
    data_buffer = '' # to store each character
    i = line + ((line-1)*25) # enables smart line jumping... tho sensitive...
    while (True):
        if big_string[i] == '*': # only return after it meets '*'
            return data_buffer + '*'
        data_buffer += big_string[i]
        i += 1

`i` is the index to which the while loop will **start** adding data from the `big_string` to the `data_buffer`. `i` will be initialized with the formula `line + ((line-1)*25)` which will be demonstrated below :
```python
line = 1 -> i = 1 + ((1-1)*25) = 1 + (0*25) = 1   # here are indexes of the START of each line in the big_string
line = 2 -> i = 2 + ((2-1)*25) = 2 + (1*25) = 27  # 1, 27, 53 are 25 characters apart, a linear function that tells the start index of each line
line = 3 -> i = 3 + ((3-1)*25) = 3 + (2*25) = 53  # 1 belongs to line 1, 27 belongs to line 2, 53 belongs to line 3, and so on...
```

Each while loop iteration, `i` will increment by 1 to move to the next character in the `big_string` as it stores each character one by one into `data_buffer`. The loop continues until it encounters **the first asterisk `*`, which signifies the end of the relevant data** for that line. The function then returns the collected data in `data_buffer` being a `string` of the line of product data fields with its separators.

---

# read_data(product_line, separator1, separator2)
where `product_line` is a `string` from the line data fields of the big_string. **`product_line` is obtained with the help of the `get_line(line, big_string)` function**. `separator1` and `separator2` are the two separators that **surround the data field to be extracted**, such as `-` and `.` for the product name, or `.` and `:` for the product price in IDR, and so on.
> This function extracts and returns a `string` which is the **data field from `product_line` that is located between `separator1` and `separator2`.**

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> EXTRACT DATA BETWEEN 2 SEPARATORS
def read_data(product_line, separator1, separator2):
    extracted_data = ''
    x = 0 # this is just to count string index
    for i in product_line:
        if i == separator1: # START EXTRACTION
            x += 1 # skip the first separator1
            while product_line[x] != separator2: # EXTRACT UNTIL product[x] is the second separator2
                extracted_data += product_line[x]
                x += 1
            # After successful data extraction, RETURN
            return extracted_data
        x += 1 # keep looping until the first seperator1 is found

`x` is initialized at 0 as an index to keep track of which string index of `product_line` the for loop is currently examining. The for loop iterates through the first few characters of `product_line` until `i` is the first occurrence of `separator1`. Once found, `x` is incremented by one to skip that first separator. Then it starts the while loop that begins extracting characters from `product_line` with that `x` index, adding each character to `extracted_data` until it encounters `separator2`. The function then returns the collected `extracted_data`. 

---
# write_data(product_line, separator1, value, separator2)
where `product_line` is a `string` from the line data fields of the big_string. **`product_line` is obtained with the help of the `get_line(line, big_string)` function**. `separator1` and `separator2` are the two separators that **surround the data field to be replaced or inserted**, and `value` is the new data to be inserted between the two separators.
> This function returns a `string` which is the **updated `product_line` with the `value` inserted between `separator1` and `separator2`, replacing any existing data in that field.**

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> WRITE DATA BETWEEN 2 SEPARATORS
def write_data(product_line, separator1, value, separator2):
    updated_product = '' # a buffer for the updated product
    value = str(value) # make sure value is a string

    x = 0 # this is just to count string index
    # This for loop is just for copying the chars before seperator1
    # Until it finds separator 1 then proceed injecting
    for i in product_line:
        if i == separator1: # START WRITING
            updated_product += i + value # add the separator1 i then add the whole value
            # This skips the characters that are replaced by the value
            while product_line[x] != separator2:
                x += 1
            # add the rest of the original data that is unchanged
            while product_line[x] != '*':
                updated_product += product_line[x]
                x += 1
            # After successful writing, BREAK out of the for loop, no longer needed
            break
        else:
            updated_product += i
        x += 1

    # check if the updated product became more than the 25 char limit
    length = len(updated_product)
    if length > 25:
        print(updated_product)
        float("VALUE IS TOO LARGE, THE WHOLE PRODUCT LINE EXCEEDS 25")
    # after writing the updated_product can be less than 25 chars so gotta add fillers
    updated_product = updated_product + ('*' * (25 - length)) if length < 25 else updated_product
    return updated_product + '\n' # FINALLY

`updated_product` is initialized as an empty string to serve as a buffer for the updated product line. `value` is converted to a string to ensure it can be concatenated with other string components. `x` is initialized at 0 to keep track of the current index in `product_line`.   

Next, a for loop iterates through each character `i` in `product_line`. **When `i` matches `separator1`, it indicates the start of the section to be replaced**. The function appends `separator1` and the new `value` to `updated_product`. A while loop then increments `x` to skip over the existing data until it reaches `separator2`. After that, another while loop appends the remaining characters from `product_line` starting from `separator2` until it encounters an asterisk `*`, which signifies the end of the relevant data. Finally, the function breaks out of the for loop and **returns the fully constructed `updated_product` `string` that builds one line of a big_string**.

## get_line(), read_data(), write_data() functions are the foundational functions that manage the big string structures in this program. These functions allow for modular and flexible management of product data, enabling easy addition, removal, and updating of products in the vending machine's inventory.